In [ ]:
import os
import h5py
import mahotas as mh
import numpy as np

from skimage import measure
from stl import mesh



import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline

In [ ]:
DATA = '/home/d/data/ac3x75/mojo/ids/tiles/w=00000000/'


In [ ]:
def threshold(arr, val):
    out = np.zeros((arr.shape[0], arr.shape[1]), dtype=np.bool)
    out[arr == val] = 1
    return out

In [ ]:
def store_mesh(arr, filename):
    
    verts, faces = measure.marching_cubes(arr, 0, spacing=(1.,1.,1.),gradient_direction='ascent')
    applied_verts = verts[faces]
    
    mesh_data = np.zeros(applied_verts.shape[0], dtype=mesh.Mesh.dtype)
    
    print 'Converting', applied_verts.shape[0], 'vertices'
    for i, v in enumerate(applied_verts):
        mesh_data[i][1][0] = v[0]
        mesh_data[i][1][1] = v[1]
        mesh_data[i][1][2] = v[2]
        
    m = mesh.Mesh(mesh_data)
    with open(filename, 'w') as f:
        m.save(filename, f)
        
    print 'Saved.'
    
    return m

In [ ]:
NEURON_ID = 3036
# X_SHAPE = 8192
# Y_SHAPE = 8192
X_SHAPE=1024
Y_SHAPE=1024
Z_SHAPE = (0,75)
SPLINE_RESOLUTION = 1/16.

thresholded_3d = np.zeros((Z_SHAPE[1], Y_SHAPE, X_SHAPE), dtype=np.bool)
# threed = np.zeros((Z_SHAPE[1], Y_SHAPE, X_SHAPE), dtype=np.uint64)


for SLICE in range(Z_SHAPE[0], Z_SHAPE[1]):

    img = np.zeros((Y_SHAPE,X_SHAPE), dtype=np.uint64)
    tiles = sorted(os.listdir(os.path.join(DATA, 'z='+str(SLICE).zfill(8))))
    
    
    for t in tiles:
        
        if t.startswith('.'):
            continue
        
        filepath = os.path.join(DATA, 'z='+str(SLICE).zfill(8), t)
        y = int(t.split(',')[0].split('=')[1])
        x = int(t.split(',')[1].split('=')[1].split('.')[0])
        with h5py.File(filepath, 'r') as f:
            data = f.get('IdMap')
            img[y*512:y*512+512, x*512:x*512+512] = data

    # now threshold this bad boy
    thresholded_slice = threshold(img, NEURON_ID)
    thresholded_3d[SLICE] = thresholded_slice
#     threed[SLICE] = img
    
    print 'loaded', SLICE
            

In [ ]:
imshow(thresholded_3d[0])

In [ ]:
upsampled = thresholded_3d.repeat(10, axis=0)

In [ ]:
upsampled.shape

In [ ]:
import math
from skimage import measure
import scipy.spatial as kd

def relabel(array):
    return measure.label(array+1).astype(np.uint64)-1

def borderer(relabeled):
    big_spot = np.zeros(relabeled.shape, dtype=np.bool)
    return mh.labeled.borders(big_spot, edge)
    

In [ ]:

resliced = upsampled.swapaxes(0,1)
spots = relabel(resliced[0])

edges = measure.find_contours(spots, 0) 
y,x = zip(*edges[0])
#plt.plot(x, y, 'o')

# get the cumulative distance along the contour
dist = np.sqrt((np.diff(x))**2 + (np.diff(y))**2).cumsum()[-1]
# build a spline representation of the contour
spline, u = scipy.interpolate.splprep([x, y], s=0)
sampler = np.linspace(0, u[-1], SPLINE_RESOLUTION*dist)

# resample it at smaller distance intervals
interp_x, interp_y = scipy.interpolate.splev(sampler, spline)
ix,iy = [np.array(i,dtype=int) for i in [interp_x,interp_y]]
edge_image = np.zeros(spots.shape,dtype=int)

# plt.plot(x, y, 'o')
# plt.plot(ix, iy, '-')
for j in range(1,len(iy)):
    mh.polygon.line((iy[j-1], ix[j-1]), (iy[j], ix[j]), edge_image)
plt.figure(figsize=(10,10))
imshow(edge_image)



In [ ]:
from numpy import random, meshgrid, arange
from scipy.ndimage import map_coordinates

(nx, ny, nz) = (300, 300, 750)
A = upsampled[:,0:300,0:300].swapaxes(0,2)

# random floating-point indices in [0, nz-1]
Z = random.rand(nx, ny)*(nz-1)

# regular integer indices of shape (nx,ny)
X, Y = meshgrid(arange(nx), arange(ny), indexing='ij')

coords = (X, Y, Z) # X, Y, and Z are of shape (nx, ny)

len(map_coordinates(A, coords, order=1, cval=-999.))

In [ ]:
all_borders = 0

for z in range(upsampled.shape[0]):
    

    all_borders += mh.labeled.borders(filtered[z])
    
plt.figure(figsize=(10,10))
imshow(all_borders)

In [ ]:
all_borders = 0

for z in range(upsampled.shape[0]):
    

    all_borders += mh.labeled.borders(hanning[z])
    
plt.figure(figsize=(10,10))
imshow(all_borders)

In [ ]:
m1 = store_mesh(thresholded_3d, '/home/d/data/neurons/'+str(NEURON_ID)+'.stl')

In [ ]:
m1 = store_mesh(filtered, '/home/d/data/neurons/'+str(NEURON_ID)+'_smooth.stl')

In [ ]:
from scipy.interpolate import griddata

In [ ]:
grid_z = griddate((thresholded_3d[0], thresholded_3d[1], thresholded_3d[2], thresholded_3d[3]), )